<a href="https://colab.research.google.com/github/fvillena/dcc-ia-nlp/blob/master/3-svd_naive_bayes.sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SVD y Naïve Bayes

Preparamos el ambiente

In [1]:
!wget https://files.fast.ai/data/examples/imdb_sample.tgz
!tar -xvf imdb_sample.tgz

--2021-11-22 00:08:12--  https://files.fast.ai/data/examples/imdb_sample.tgz
Resolving files.fast.ai (files.fast.ai)... 104.26.2.19, 172.67.69.159, 104.26.3.19, ...
Connecting to files.fast.ai (files.fast.ai)|104.26.2.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 571827 (558K) [application/x-gtar-compressed]
Saving to: ‘imdb_sample.tgz.1’

imdb_sample.tgz.1   100%[===================>] 558.42K  --.-KB/s    in 0.1s    

2021-11-22 00:08:12 (3.81 MB/s) - ‘imdb_sample.tgz.1’ saved [571827/571827]

imdb_sample/
imdb_sample/texts.csv


In [2]:
import pandas as pd
import sklearn.datasets
import sklearn.feature_extraction
import sklearn.decomposition
import numpy as np

In [3]:
remove = ('headers', 'footers', 'quotes')
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups = sklearn.datasets.fetch_20newsgroups(remove=remove,categories=categories)
tfidf_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(stop_words="english",min_df=0.005)
x = tfidf_vectorizer.fit_transform(newsgroups.data)

## Actividad 1: SVD
Desde la matriz de tf-idf calculada anteriormente sobre newsgroups realice una descomposición mediante SVD.

1.   Verifique que las matrices U y V calculadas con SVD son ortonormales
2.   Verifique hasta que vector los temas identificados en SVD hacen “sentido” para los 4 temas seleccionados.
3.   ¿Cuántos temas potencialmente se pueden inferir con SVD?

In [4]:
## HINT: Estas lineas de código pueden serle útiles para resolver la actividad
import numpy as np
vocab = tfidf_vectorizer.get_feature_names()
def show_topics(a):
  num_top_words=8
  top_words_values = lambda t: [f"{vocab[i]}({str(round(v,3))})" for i,v in zip(np.argsort(t)[:-num_top_words-1:-1],np.sort(t)[:-num_top_words-1:-1])]
  topic_words_values = ([top_words_values(t) for t in a])
  return [' '.join(t) for t in topic_words_values]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [5]:
#SVD
U, s, Vh = np.linalg.svd(x.toarray(), full_matrices=False)

In [6]:
product_u = U.dot(U.T)
np.fill_diagonal(product_u,0)
np.allclose(product_u, 0, atol=1e-08, equal_nan=False)

True

In [7]:
product_v = Vh.dot(Vh.T)
np.fill_diagonal(product_v,0)
np.allclose(product_v, 0, atol=1e-08, equal_nan=False)

True

In [8]:
np.allclose(np.sum(np.abs(U)**2,axis=-1)**(1./2), 1, atol=1e-08, equal_nan=False)

True

In [9]:
np.allclose(np.sum(np.abs(Vh)**2,axis=-1)**(1./2), 1, atol=1e-08, equal_nan=False)

True

In [10]:
# palabras y valores asociados a los tópicos
show_topics(Vh)[:10]

['contributed(-0.002) extensive(-0.002) union(-0.002) 85(-0.002) aeronautics(-0.002) overall(-0.002) concluded(-0.002) 43(-0.002)',
 'god(0.357) jesus(0.168) people(0.135) bible(0.124) believe(0.099) christian(0.096) say(0.093) religion(0.08)',
 'thanks(0.246) graphics(0.172) files(0.151) god(0.13) file(0.128) image(0.114) does(0.113) windows(0.112)',
 'god(0.582) space(0.234) jesus(0.207) bible(0.119) edu(0.117) nasa(0.087) satan(0.085) christ(0.074)',
 'jesus(0.063) does(0.059) bible(0.048) objective(0.045) christian(0.043) morality(0.037) think(0.035) moral(0.032)',
 'image(0.237) graphics(0.176) file(0.139) data(0.12) use(0.104) images(0.091) software(0.087) computer(0.083)',
 'jesus(0.186) just(0.145) koresh(0.117) thanks(0.1) graphics(0.092) don(0.082) like(0.076) bible(0.076)',
 'files(0.308) file(0.207) format(0.163) cview(0.133) program(0.117) image(0.117) know(0.107) tiff(0.104)',
 'mode(0.143) card(0.125) lunar(0.113) moon(0.109) software(0.109) orbit(0.102) need(0.093) time

In [11]:
print(newsgroups.data[30])


  Actually, my atheism is based on ignorance.  Ignorance of the
  existence of any god.  Don't fall into the "atheists don't believe
  because of their pride" mistake.


/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\ 

Bob Beauchaine bobbe@vice.ICO.TEK.COM 

They said that Queens could stay, they blew the Bronx away,
and sank Manhattan out at sea.


In [12]:
U[30,:]

array([-2.19457759e-02,  3.34805569e-02,  7.31009657e-03, ...,
       -5.34386730e-19,  0.00000000e+00,  1.25386537e-16])

In [13]:
np.argsort(U[30,:])

array([   4, 1919, 1920, ..., 1915, 1951, 1929])

In [14]:
show_topics(Vh)[4]

'jesus(0.063) does(0.059) bible(0.048) objective(0.045) christian(0.043) morality(0.037) think(0.035) moral(0.032)'

Para este paso práctico utilizaremos el conjunto de datos de imbd en donde tenemos una serie de documentos etiquetados con la polaridad (negativo o positivo) asociada.

In [15]:
imdb = pd.read_csv("imdb_sample/texts.csv")

In [16]:
imdb

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False
...,...,...,...
995,negative,There are many different versions of this one ...,True
996,positive,Once upon a time Hollywood produced live-actio...,True
997,negative,Wenders was great with Million $ Hotel.I don't...,True
998,negative,Although a film with Bruce Willis is always wo...,True


Para construir nuestra matriz de terminos y documentos utilizademos la clase CountVectorizer de sklearn.

In [17]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(stop_words="english")
term_doc_matrix_train = count_vectorizer.fit_transform(imdb[imdb.is_valid == False].text)

Nuestra matriz de términos y documentos tiene la forma (documentos,términos)

In [18]:
term_doc_matrix_train

<800x16163 sparse matrix of type '<class 'numpy.int64'>'
	with 74815 stored elements in Compressed Sparse Row format>

Calculamos la cantidad de apariciones de cada una de las palabras en cada una de las clases, junto con la cantidad de apariciones de cada clase.

In [19]:
c_w_negative = np.asarray(term_doc_matrix_train.todense()[imdb[imdb.is_valid == False].label == "negative",:].sum(0)).reshape(-1) # apariciones de cada una de las palabras en la clase negativa
c_w_positive = np.asarray(term_doc_matrix_train.todense()[imdb[imdb.is_valid == False].label == "positive",:].sum(0)).reshape(-1)
c_negative = (imdb[imdb.is_valid == False].label == "negative").sum() # apariciones de la clase negativa
c_positive = (imdb[imdb.is_valid == False].label == "positive").sum()

## Actividad 2: Palabras más comunes por clase

Dados los arreglos que contienen la cantidad de apariciones de cada palabra en cada clase, imprima las 3 palabras más comunes para cada clase.

In [20]:
vocab = count_vectorizer.get_feature_names()
print(np.array(vocab)[np.argsort(c_w_negative)][-3:])
print(np.array(vocab)[np.argsort(c_w_positive)][-3:])

['film' 'movie' 'br']
['movie' 'film' 'br']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Actividad 3: Ratio positivo - negativo

Calcule el ratio positivo negativo en esta base de datos, y muestre cuales son las 3 palabras más probables en cada clase.

In [21]:
p_w_negative = (c_w_negative + 1) / c_negative # Probabilidad asociada a cada palabra en la clase negativa
p_w_positive = (c_w_positive + 1) / c_positive 

In [22]:
# Programa
r = np.log(p_w_positive / p_w_negative)
print(np.array(vocab)[np.argsort(r)][:20])
print(np.array(vocab)[np.argsort(r)][-20:])

['porn' 'vargas' 'crap' 'naschy' 'dog' 'crater' 'worst' 'disappointment'
 'fuqua' 'soderbergh' 'scorsese' 'johnson' 'jackman' 'poorly' 'junior'
 'rangers' 'lake' 'bey' 'bet' 'dinosaur']
['steals' 'arnold' 'damon' '3d' 'hanzo' 'alfred' 'cheech' 'howard' 'sport'
 'paxton' 'gilliam' 'jabba' 'davies' 'han' 'felix' 'jimmy' 'noir'
 'astaire' 'fanfan' 'biko']


## Actividad 4: Clasificador

Ahora calcule el log ratio de cada documento. Si asumimos que > 0 es opinión positiva, ¿cuál es el accuracy de la clasificación?

In [23]:
# Programa
# term_doc_matrix_train.shape
term_doc_matrix_test = count_vectorizer.transform(imdb[imdb.is_valid == True].text)
test_pred = term_doc_matrix_test.dot(r) > 0
test_label = np.where(imdb[imdb.is_valid == True].label == "positive", True, False)
confusion_matrix = pd.crosstab(test_label,test_pred)
print(np.array(confusion_matrix).diagonal().sum() / np.array(confusion_matrix).sum())

0.765


## Actividad 5: Bigramas

Genere un nuevo clasificador, esta vez agregando bigramas al vocabulario de la matriz de términos y documentos.

In [24]:
# Programa
count_vectorizer_bigrams = sklearn.feature_extraction.text.CountVectorizer(stop_words="english",ngram_range=(1,2))
term_doc_matrix_train_bigrams = count_vectorizer_bigrams.fit_transform(imdb[imdb.is_valid == False].text)
c_w_negative_bigrams = np.asarray(term_doc_matrix_train_bigrams.todense()[imdb[imdb.is_valid == False].label == "negative",:].sum(0)).reshape(-1) # apariciones de cada una de las palabras en la clase negativa
c_w_positive_bigrams = np.asarray(term_doc_matrix_train_bigrams.todense()[imdb[imdb.is_valid == False].label == "positive",:].sum(0)).reshape(-1)
p_w_negative_bigrams = (c_w_negative_bigrams + 1) / c_negative # Probabilidad asociada a cada palabra en la clase negativa
p_w_positive_bigrams = (c_w_positive_bigrams + 1) / c_positive 
r_bigrams = np.log(p_w_positive_bigrams / p_w_negative_bigrams)
term_doc_matrix_test_bigrams = count_vectorizer_bigrams.transform(imdb[imdb.is_valid == True].text)
test_pred_bigrams = term_doc_matrix_test_bigrams.dot(r_bigrams) > 0
confusion_matrix_bigrams = pd.crosstab(test_label,test_pred_bigrams)
print(np.array(confusion_matrix_bigrams).diagonal().sum() / np.array(confusion_matrix_bigrams).sum())

0.77
